In [ ]:
import os
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType
import pandas

In [ ]:
storage_name = os.getenv("DLS_NAME")
filesystem_stage = os.getenv("DLS_FILESYSTEM_STAGE")
tenant_id = os.getenv("SP_TENANT_ID")
application_id = os.getenv("SP_CLIENT_ID")

service_credential = dbutils.secrets.get(scope="keyvault-managed", key="dlsserviceprincipalsecret")

spark.conf.set(f"fs.azure.account.auth.type.{storage_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_name}.dfs.core.windows.net", application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_name}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [ ]:
lake_url = f"abfss://{filesystem_stage}@{storage_name}.dfs.core.windows.net"

# https://spark.apache.org/docs/latest/sql-data-sources-csv.html
loader = spark.read.format("CSV")\
    .option("header", "true")\
        .option("inferSchema", "true")\
            .option("escape", "\\")\
                .option("delimiter", ",")

athletes = loader.load(f"{lake_url}/Athletes.xlsx.csv")
coaches = loader.load(f"{lake_url}/Coaches.xlsx.csv")
entries_gender = loader.load(f"{lake_url}/EntriesGender.xlsx.csv")
medals = loader.load(f"{lake_url}/Medals.xlsx.csv")
teams = loader.load(f"{lake_url}/Teams.xlsx.csv")

In [ ]:
display(medals)

athletes.printSchema()
entries_gender.printSchema()
medals.printSchema()

In [ ]:
entries_gender = entries_gender.withColumn("Female", col("Female").cast(IntegerType()))\
    .withColumn("Male", col("Male").cast(IntegerType()))\
    .withColumn("Total", col("Total").cast(IntegerType()))

In [ ]:
top_gold_medal_countries = medals.orderBy("Gold", ascending=False).select("Team/NOC", "Gold").show()

In [ ]:
avg_entries_by_gender = entries_gender.withColumn(
    'Avg_Female', entries_gender['Female'] / entries_gender['Total']
).withColumn(
    'Avg_Male', entries_gender['Male'] / entries_gender['Total']
)
avg_entries_by_gender.show()

In [ ]:
athletes_pandas = athletes.toPandas()
athletes_pandas.to_csv(f"abfss://transformed-data@{storage_name}.dfs.core.windows.net/athletes.csv")